### Phenotype Table I
December 2019
#### Traits
* Season 4 `end_of_season_biomass`
* Season 4 `growing_degree_days`

#### Notes
* one plot per row

In [1]:
import datetime
import pandas as pd
import numpy as np

In [12]:
df_0 = pd.read_csv('../data/raw/mac_season_4.csv', low_memory=False)
print(df_0.shape)
# df_0.head()

(372363, 39)


### I. Reset index to sitename 
* Maintain shape of dataframe
* No grouping
* Strip `E` and `W` from sitenames

In [11]:
e_w_subplots = df_0.loc[(df_0.sitename.str.endswith(' E')) | (df_0.sitename.str.endswith(' W'))]
e_w_subplots.shape

(9460, 39)

In [31]:
new_sitenames = []
e_w_counter = 0

for site in df_0.sitename.values:
    
    if (site.endswith(' W')) | (site.endswith(' E')):
        new_name = site[:-2]
        new_sitenames.append(new_name)
        e_w_counter += 1
        
    else:
        new_sitenames.append(site)
        
print(f'Length of original sitename values list: {len(df_0.sitename.values)}')
print(f'Length new sitename list: {len(new_sitenames)}')
print(f'Number of E and W subplots: {e_w_counter}')

Length of original sitename values list: 372363
Length new sitename list: 372363
Number of E and W subplots: 9460


In [32]:
df_0['plot_name'] = new_sitenames
print(df_0.shape)
df_0.head()

(372363, 40)


,Unnamed: 0,checked,result_type,id,citation_id,site_id,treatment_id,sitename,city,lat,...,statname,stat,notes,access_level,cultivar,entity,method_name,view_url,edit_url,plot_name
0,1,0,traits,6001958927,6.000000e+09,6000005673,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 5,Maricopa,33.074907,...,NaN,NaN,NaN,2,PI181083,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...,MAC Field Scanner Season 4 Range 11 Column 5
1,2,0,traits,6001958928,6.000000e+09,6000005676,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 6,Maricopa,33.074907,...,NaN,NaN,NaN,2,PI564163,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...,MAC Field Scanner Season 4 Range 11 Column 6
2,3,0,traits,6001958931,6.000000e+09,6000005685,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 9,Maricopa,33.074907,...,NaN,NaN,NaN,2,PI52606,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...,MAC Field Scanner Season 4 Range 11 Column 9
3,4,0,traits,6001958933,6.000000e+09,6000005691,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 11,Maricopa,33.074907,...,NaN,NaN,NaN,2,PI533792,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...,MAC Field Scanner Season 4 Range 11 Column 11
4,5,0,traits,6001958936,6.000000e+09,6000005700,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 14,Maricopa,33.074907,...,NaN,NaN,NaN,2,PI535794,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...,MAC Field Scanner Season 4 Range 11 Column 14


In [33]:
df_1 = df_0.set_index(keys='plot_name')
print(df_1.shape)
df_1.head()

(372363, 39)


,Unnamed: 0,checked,result_type,id,citation_id,site_id,treatment_id,sitename,city,lat,...,n,statname,stat,notes,access_level,cultivar,entity,method_name,view_url,edit_url
plot_name,,,,,,,,,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 11 Column 5,1,0,traits,6001958927,6.000000e+09,6000005673,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 5,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI181083,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
MAC Field Scanner Season 4 Range 11 Column 6,2,0,traits,6001958928,6.000000e+09,6000005676,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 6,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI564163,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
MAC Field Scanner Season 4 Range 11 Column 9,3,0,traits,6001958931,6.000000e+09,6000005685,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 9,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI52606,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
MAC Field Scanner Season 4 Range 11 Column 11,4,0,traits,6001958933,6.000000e+09,6000005691,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 11,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI533792,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
MAC Field Scanner Season 4 Range 11 Column 14,5,0,traits,6001958936,6.000000e+09,6000005700,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 14,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI535794,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...


In [36]:
print(f'Number of unique plot names: {df_1.index.nunique()}')

Number of unique plot names: 847


### II. Aboveground dry biomass (end of season)

#### Check for null `aboveground_dry_biomass` values

In [38]:
bio_values_only = df_1.loc[df_1.trait == 'aboveground_dry_biomass']
print(f'Number of aboveground dry biomass values in dataset: {bio_values_only.shape[0]}')
print(f'Number of unique plots in biomass df: {bio_values_only.index.nunique()}')

Number of aboveground dry biomass values in dataset: 200
Number of unique plots in biomass df: 199


In [50]:
# Identify the duplicate plot

duplicates = bio_values_only[bio_values_only.index.duplicated(keep=False)]
# duplicates

,Unnamed: 0,checked,result_type,id,citation_id,site_id,treatment_id,sitename,city,lat,...,n,statname,stat,notes,access_level,cultivar,entity,method_name,view_url,edit_url
plot_name,,,,,,,,,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 20 Column 11,72499,0,traits,6001966414,6.000000e+09,6000005428,6.000000e+09,MAC Field Scanner Season 4 Range 20 Column 11,Maricopa,33.075231,...,NaN,NaN,NaN,NaN,4,PI527045,NaN,Whole above ground biomass at harvest,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
MAC Field Scanner Season 4 Range 20 Column 11,279789,0,traits,6001966414,6.000000e+09,6000005428,6.000000e+09,MAC Field Scanner Season 4 Range 20 Column 11,Maricopa,33.075231,...,NaN,NaN,NaN,NaN,4,PI527045,NaN,Whole above ground biomass at harvest,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...


In [76]:
print(duplicates.index.unique())
print(duplicates['mean'].unique())

Index(['MAC Field Scanner Season 4 Range 20 Column 11'], dtype='object', name='plot_name')
[10950.]


In [67]:
non_duplicate_columns = []

for col in duplicates.columns:
    
    if duplicates.iloc[0][col] != duplicates.iloc[1][col]:
        non_duplicate_columns.append(col)
        
# print(non_duplicate_columns)

for col in non_duplicate_columns:
    print(f'{col}: {duplicates.iloc[0][col]}')
    print(f'{col}: {duplicates.iloc[1][col]}')

Unnamed: 0: 72499
Unnamed: 0: 279789
n: nan
n: nan
statname: nan
statname: nan
stat: nan
stat: nan
notes: nan
notes: nan
entity: nan
entity: nan


Since the `aboveground_dry_biomass` value and plot name are duplicated, one of these rows can be dropped from the dataset.

In [80]:
df_1.loc[(df_1.index == 'MAC Field Scanner Season 4 Range 20 Column 11') & (df_1['mean'] == 10950)]

,Unnamed: 0,checked,result_type,id,citation_id,site_id,treatment_id,sitename,city,lat,...,n,statname,stat,notes,access_level,cultivar,entity,method_name,view_url,edit_url
plot_name,,,,,,,,,,,,,,,,,,,,,
MAC Field Scanner Season 4 Range 20 Column 11,72499,0,traits,6001966414,6.000000e+09,6000005428,6.000000e+09,MAC Field Scanner Season 4 Range 20 Column 11,Maricopa,33.075231,...,NaN,NaN,NaN,NaN,4,PI527045,NaN,Whole above ground biomass at harvest,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
MAC Field Scanner Season 4 Range 20 Column 11,279789,0,traits,6001966414,6.000000e+09,6000005428,6.000000e+09,MAC Field Scanner Season 4 Range 20 Column 11,Maricopa,33.075231,...,NaN,NaN,NaN,NaN,4,PI527045,NaN,Whole above ground biomass at harvest,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...


#### Create new column `aboveground_dry_biomass` and populate with values

In [ ]:
for index, row in df_5.iterrows():            
        if counter % 1000 == 0:
            print(counter)            
            counter += 1
        for trait in traits_to_keep:
            if row['trait'] == trait:                
                df_5.loc[index, [trait]] = row['value']

In [4]:
print(f'Shape of original dataframe: {df_0.shape}')
print(f'Shape of re-indexed dataframe: {df_1.shape}')

Shape of original dataframe: (372363, 39)
Shape of re-indexed dataframe: (372363, 38)


#### II. Determine number of rows for processed dataset
* Unique sitenames
* Strip ` E` and ` W` from sitenames

In [6]:
print(f'Number of unique sitenames with E and W subplots: {df_1.index.nunique()}')

Number of unique sitenames with E and W subplots: 2202


#### Extract Range and Column Values

In [17]:
df_4 = df_3.copy()

In [19]:
df_4['range'] = df_4['sitename'].str.extract("Range (\d+)").astype(int)
df_4['column'] = df_4['sitename'].str.extract("Column (\d+)").astype(int)

In [31]:
# intentional csv creation

do_you_need_to_create_another_csv = False

if do_you_need_to_create_another_csv:

    timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
    output_filename = f'aboveground_biomass_harvest_{timestamp}.csv'.replace(':', '')
    df_5.to_csv(f'../data/processed/{output_filename}')